配置环境，第一次运行时执行

In [1]:
from IPython.display import clear_output

!pip install -U openmim
!mim install git+https://github.com/open-mmlab/mmengine.git  # 0.7.2
!mim install "mmcv==2.0.0"

!git clone https://github.com/ZJLauu/mmsegmentation.git -b bi_she
%cd mmsegmentation
!pip install -e.
%cd ..

clear_output()

验证环境

In [2]:
import torch
print(torch.__version__, torch.cuda.is_available())
 
# Check MMSegmentation installation
import mmseg
print(mmseg.__version__)
 
# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

c:\Users\ldp\.conda\envs\mm2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1.13.0+cu116 True


ModuleNotFoundError: No module named 'mmseg'

In [2]:
!nvidia-smi

Tue Apr 25 15:06:00 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.89.02    Driver Version: 525.89.02    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:56:00.0 Off |                  N/A |
| 30%   26C    P8    35W / 350W |      2MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from mmengine import Config
from mmengine.runner import Runner

# train

In [ ]:
cfg = Config.fromfile('/root/autodl-tmp/configs/Tr_base2.0.py')

cfg.data_root = '/root/autodl-tmp/Trans10K_cls12'
cfg.experiment_name = f'SegTrue_1.5e-5_160k_b8_0.75-1.5-crop512-512'
cfg.work_dir = f'/root/autodl-tmp/work_dir/{cfg.experiment_name}'

# cfg.resume = True
# cfg.load_from = '/root/autodl-tmp/work_dir/SegTru_1.5e-5_160k_b8_0.75-1.5-crop512-512/iter_146000.pth'
cfg.load_from = None
# cfg.custom_hooks[0].resume = cfg.resume  # ema

cfg.visualizer.save_dir = cfg.work_dir

cfg.train_dataloader.batch_size = 8

cfg.optim_wrapper.optimizer.lr = 1.5e-5

cfg.train_cfg.max_iters = 160000
cfg.train_cfg.val_interval = cfg.default_hooks.checkpoint.interval = int(cfg.train_cfg.max_iters / 80 )
cfg.default_hooks.logger.interval= int(cfg.train_cfg.val_interval / 8)

cfg.param_scheduler[0].start_factor = 1e-6
cfg.param_scheduler[0].end = int(cfg.train_cfg.max_iters / 100)  # warm-up steps
cfg.param_scheduler[1].begin = cfg.param_scheduler[0].end
cfg.param_scheduler[1].end = cfg.train_cfg.max_iters
cfg.default_hooks.logger.interval_exp_name = cfg.train_cfg.val_interval
# cfg.custom_hooks[0].begin_iter = cfg.param_scheduler[1].begin  # ema

print(f'Config:\n{cfg.pretty_text}')

In [ ]:
runner = Runner.from_cfg(cfg)
runner.train()

# test

In [ ]:
import os
from mmengine import Config
from mmengine.runner import Runner
import warnings
warnings.filterwarnings("ignore")
work_dir = 'work_dir/SegTrue_1.5e-5_160k_b8_0.75-1.5-crop512-512'
ckpt = 'best_mIoU_iter_116000.pth'

test_cfg = Config.fromfile(f'{work_dir}/Tr_base2.0.py')
test_cfg.resume = False
test_cfg.load_from = os.path.join(work_dir, ckpt)
test_cfg.visualizer.save_dir = test_cfg.work_dir = os.path.join(work_dir, 'test')

test_runner = Runner.from_cfg(test_cfg)
test_runner.test()